<a href="https://colab.research.google.com/github/DCajiao/Aplicaciones-del-Analisis-Espacial/blob/main/casos/Situaci%C3%B3n_2/00_preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Carga Inicial de Datos**

In [237]:
# Librerías y Dataset

import requests
import pandas as pd
import numpy as np
from io import BytesIO
import re
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [238]:

def summary_by_columns(df):
    """
    Generates a summary of each column in the provided DataFrame.

    The summary includes the data type, number of missing values, number of unique values,
    and number of duplicate values for each column.

    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame to summarize.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing the summary information for each column.
    """
    # Create an empty DataFrame to store the summary
    summary_df = pd.DataFrame(
        columns=['Column', 'Data Type', 'Missing Values', 'Unique Values', 'Duplicates'])

    # Loop through each column in the DataFrame
    for col in df.columns:
        data_type = df[col].dtype
        missing_values = df[col].isna().sum()
        num_unique_values = df[col].nunique()
        num_duplicates = df[col].duplicated().sum()

        # Add the results to the summary DataFrame
        row_summary = pd.DataFrame({
            'Column': [col],
            'Data Type': [data_type],
            'Missing Values': [missing_values],
            'Unique Values': [num_unique_values],
            'Duplicates': [num_duplicates],
            'Missing Values (%)': [round((missing_values / df.shape[0]) * 100, 2)]
        })

        # Concatenate the row summary to the summary DataFrame
        summary_df = pd.concat([summary_df, row_summary], ignore_index=True)

    return summary_df

In [239]:
# Importación de Datos
DATA_GITHUB_URL = f'https://raw.githubusercontent.com/DCajiao/Aplicaciones-del-Analisis-Espacial/refs/heads/main/casos/Situación_2/data/input/AccidentesFatales'

df_2009 = pd.read_excel(BytesIO(requests.get(f'{DATA_GITHUB_URL}_2009.xlsx').content), engine='openpyxl')
df_2010 = pd.read_excel(BytesIO(requests.get(f'{DATA_GITHUB_URL}_2010.xlsx').content), engine='openpyxl')

#### **Concatenación de Datos**

Buscar la manera de hacer un merge de los datos con la etiqueta del año de origen para así mismo hacer un sólo EDA de ambos años

In [240]:
# Tienen las mismas columnas?
print(df_2009.columns)
print(df_2010.columns)

print("-"*90)

print(set(df_2009.columns) - set(df_2010.columns))
print(set(df_2010.columns) - set(df_2009.columns))

Index(['MES FALLECIMIENTO', 'MES ACCIDENTE', 'BARRIO', 'COM', 'PROFESION',
       'SEXO', 'EDAD', 'EDAD AGRUPADA', 'FECHA ACCIDENTE',
       'FECHA FALLECIMIENTO', 'HORA FALLECIMIENTO', 'HORA ACCIDENTE',
       'DIA SEMANA FALLECIMIENTO', 'DIA SEMANA ACCIDENTE', 'LUGAR INSP.',
       'CONDICCION', 'VEHICULOS', 'Ciudad', 'coordenada X (metros)',
       'coordenada Y (metros)', 'coordenada X (km)', 'coordenada Y (km)'],
      dtype='object')
Index(['MES FALLECIMIENTO', 'MES ACCIDENTE', 'BARRIO', 'COM', 'PROFESION',
       'SEXO', 'EDAD', 'EDAD AGRUPADA', 'FECHA ACCIDENTE',
       'FECHA FALLECIMIENTO', 'HORA FALLECIMIENTO', 'HORA ACCIDENTE',
       'DIA SEMANA FALLECIMIENTO', 'DIA SEMANA ACCIDENTE', 'FECHA INSP.',
       'LUGAR INSP.', 'CONDICCION', 'VEHICULOS', 'Ciudad',
       'coordenada X (metros)', 'coordenada Y (metros)', 'coordenada X (km)',
       'coordenada Y (km)'],
      dtype='object')
------------------------------------------------------------------------------------------

Falta la columna 'FECHA INSP'. Dado que igual no aporta información relevante para nuestro análisis, procederé a hacer el merge y eliminarla

In [241]:
# Eliminar la columna FECHA INSP.
df_2010.drop(columns=['FECHA INSP.'], inplace=True)

# Hacer un merge de los datos creando la etiqueta AÑO_DATA:
df_2009['AÑO_DATA'] = 2009
df_2010['AÑO_DATA'] = 2010

df = pd.concat([df_2009, df_2010])

In [242]:
# Tamaño final del dataset
print(f'Tamaño del dataset mezclado: {df.shape}')
print("-"*90)
# Proporción de cada AÑO_DATA
print(df['AÑO_DATA'].value_counts())
print("-"*90)
print(df['AÑO_DATA'].value_counts(normalize=True))

Tamaño del dataset mezclado: (602, 23)
------------------------------------------------------------------------------------------
AÑO_DATA
2009    322
2010    280
Name: count, dtype: int64
------------------------------------------------------------------------------------------
AÑO_DATA
2009    0.534884
2010    0.465116
Name: proportion, dtype: float64


#### **Exploración Features de Interés**

Nuestras columnas relevantes serán:
- `SEXO`
- `EDAD`
- `EDAD AGRUPADA`
- `CONDICCION`
- `coordenada X (metros)`
- `coordenada Y (metros)`
- `coordenada X (km)	`
- `coordenada Y (km)	`
- `AÑO_DATA`

In [243]:
summary_by_columns(df)

,Column,Data Type,Missing Values,Unique Values,Duplicates,Missing Values (%)
0,MES FALLECIMIENTO,object,0,12,590,0.00
1,MES ACCIDENTE,object,0,13,589,0.00
2,BARRIO,object,0,263,339,0.00
3,COM,int64,0,22,580,0.00
4,PROFESION,object,66,138,463,10.96
5,SEXO,object,2,2,599,0.33
6,EDAD,float64,10,84,517,1.66
7,EDAD AGRUPADA,object,10,18,583,1.66
8,FECHA ACCIDENTE,datetime64[ns],0,407,195,0.00
9,FECHA FALLECIMIENTO,datetime64[ns],0,393,209,0.00


In [244]:
# Eliminar nulos de SEXO y EDAD
df = df.dropna(subset=['SEXO', 'EDAD'])

In [245]:
summary_by_columns(df)

,Column,Data Type,Missing Values,Unique Values,Duplicates,Missing Values (%)
0,MES FALLECIMIENTO,object,0,12,580,0.00
1,MES ACCIDENTE,object,0,13,579,0.00
2,BARRIO,object,0,261,331,0.00
3,COM,int64,0,22,570,0.00
4,PROFESION,object,57,138,453,9.63
5,SEXO,object,0,2,590,0.00
6,EDAD,float64,0,84,508,0.00
7,EDAD AGRUPADA,object,0,18,574,0.00
8,FECHA ACCIDENTE,datetime64[ns],0,401,191,0.00
9,FECHA FALLECIMIENTO,datetime64[ns],0,389,203,0.00


In [246]:
# Mostrar registros con coordenadas nulas
df[df['coordenada X (km)'].isnull() | df['coordenada Y (km)'].isnull()]

,MES FALLECIMIENTO,MES ACCIDENTE,BARRIO,COM,PROFESION,SEXO,EDAD,EDAD AGRUPADA,FECHA ACCIDENTE,FECHA FALLECIMIENTO,HORA FALLECIMIENTO,HORA ACCIDENTE,DIA SEMANA FALLECIMIENTO,DIA SEMANA ACCIDENTE,LUGAR INSP.,CONDICCION,VEHICULOS,Ciudad,coordenada X (metros),coordenada Y (metros),coordenada X (km),coordenada Y (km),AÑO_DATA
53,MARZO,MARZO,LA MARIA,22,COMERCIANTE,F,29.0,25-29,2010-03-15,2010-03-15,16:00:00,15:10:00,LUNES,Lunes,H.U.V.,PAS. MOTO,MOTO - CAMIONETA,cali,NaN,NaN,NaN,NaN,2010
164,JULIO,JULIO,QUINTAS DE DON SIMON,17,INDEPENDIENTE,M,60.0,60-64,2010-07-02,2010-07-27,01:00:00,06:20:00,MARTES,Viernes,CL. VALLE LILI,CICLISTA,TAXI - BICICLETA,cali,NaN,NaN,NaN,NaN,2010
184,AGOSTO,AGOSTO,PORTADA,1,COMERCIANTE,M,22.0,20-24,2010-08-20,2010-08-20,11:48:00,11:48:00,VIERNES,Viernes,VIA,COND. MOTO,MOTO - CAMPERO,cali,NaN,NaN,NaN,NaN,2010


In [247]:
# Eliminar registros con coordenadas nulas
df = df.dropna(subset=['coordenada X (km)', 'coordenada Y (km)'])
summary_by_columns(df)

,Column,Data Type,Missing Values,Unique Values,Duplicates,Missing Values (%)
0,MES FALLECIMIENTO,object,0,12,577,0.00
1,MES ACCIDENTE,object,0,13,576,0.00
2,BARRIO,object,0,259,330,0.00
3,COM,int64,0,22,567,0.00
4,PROFESION,object,57,138,450,9.68
5,SEXO,object,0,2,587,0.00
6,EDAD,float64,0,84,505,0.00
7,EDAD AGRUPADA,object,0,18,571,0.00
8,FECHA ACCIDENTE,datetime64[ns],0,400,189,0.00
9,FECHA FALLECIMIENTO,datetime64[ns],0,388,201,0.00


#### **Tratamiento y Codificacion para Features de Interés**

##### ¿Por qué hay 13 meses en MES ACCIDENTE?

In [248]:
df['MES ACCIDENTE'].value_counts().sort_index()

MES ACCIDENTE
ABRIL         55
AGOSTO        49
DICIEMBRE     58
ENERO         48
FEBRERO       49
JULIO         42
JUNIO         61
MARZO         43
MAYO          53
NOVIEMBRE     50
OCTOBRE        1
OCTUBRE       34
SEPTIEMBRE    46
Name: count, dtype: int64

In [249]:
# Renombrar OCTOBRE a OCTUBRE
df['MES ACCIDENTE'] = df['MES ACCIDENTE'].replace('OCTOBRE', 'OCTUBRE')

##### Recodificación en CONDICCION

La CONDICCIÓN se refiere al vehículo en el que se encontraba la víctima

In [250]:
# Normalización
def limpiar_texto(x):
    x = x.upper().strip()
    x = re.sub(r'\s+', ' ', x)                     # espacios múltiples → uno
    x = re.sub(r'\(.*?\)', '', x)                  # elimina texto entre paréntesis
    x = re.sub(r'[^A-ZÑ0-9\s\-]', '', x)           # elimina símbolos raros
    x = x.replace('AUTO ', 'AUTOMOVIL ')           # uniformar "AUTO" → "AUTOMOVIL"
    x = x.replace('AUTO', 'AUTOMOVIL')             # reemplazo final
    x = x.replace('M OTO', 'MOTO')                 # errores tipográficos comunes
    return x.strip(' -')

df["CONDICCION"] = df["CONDICCION"].apply(limpiar_texto)


In [251]:
# ¿Cuantos vehículos únicos hay en el dataset?
print("Hay",df['CONDICCION'].nunique(),"vehículos únicos en el dataset.")
print("--"*40)
print("Las categorías son:", df['CONDICCION'].unique())

Hay 24 vehículos únicos en el dataset.
--------------------------------------------------------------------------------
Las categorías son: ['COND MOTO' 'CICLISTA' 'PEATON' 'PASAJERO' 'PAS MOTO' 'CONDUCTOR'
 'PAS AUTOMOVILMOVIL' 'PAS VOLQUETA' 'PAS BUSETA' 'PAS CAMPERO'
 'COND MOTOCARRO' 'PAS MICROBUS' 'CONDUCTOR TAXI' 'CONDUCTOR VOLQUETA'
 'CONDTAXI' 'PASAJERO AUTOMOVIL' 'COND AUTOMOVIL' 'PAS BICICLETA'
 'COND TAXI' 'PAS BUS' 'PASAJERA' 'PAS TAXI' 'PAS CAMION' 'PAS AUTOMOVIL']


In [252]:
# Diccionario de mapeo de condiciones a categorías estandarizadas
map_cond = {
    # --- Motocicleta ---
    "COND MOTO": "MOTOCICLETA",
    "PAS MOTO": "MOTOCICLETA",
    "COND MOTOCARRO": "MOTOCICLETA",

    # --- Bicicleta ---
    "CICLISTA": "BICICLETA",
    "PAS BICICLETA": "BICICLETA",

    # --- Auto ---
    "CONDUCTOR": "AUTO",
    "COND AUTO": "AUTO",
    "PAS AUTOMOVIL": "AUTO",
    "PAS AUTO": "AUTO",
    "PASAJERO AUTO": "AUTO",
    "PASAJERA": "AUTO",

    # --- Taxi (lo agrupamos en Auto) ---
    "CONDUCTOR TAXI": "AUTO",
    "COND TAXI": "AUTO",
    "CONDTAXI": "AUTO",
    "PAS. TAXI": "AUTO",

    # --- Bus ---
    "PAS BUSETA": "BUS",
    "PAS MICROBUS": "BUS",
    "PAS BUS": "BUS",
    "PASAJERO": "PEATÓN",

    # --- Camión / Volqueta / Campero (categoría Camión) ---
    "PAS VOLQUETA": "CAMIÓN",
    "CONDUCTOR VOLQUETA": "CAMIÓN",
    "PAS CAMION": "CAMIÓN",
    "PAS CAMPERO": "CAMIÓN",

    # --- Peatón ---
    "PEATON": "PEATÓN",
}

# Aplicar mapeo a la columna de condiciones
df["TIPO_AUTOMOTOR"] = df["CONDICCION"].map(map_cond).fillna("OTRO")

# Ver distribución
print(df["TIPO_AUTOMOTOR"].value_counts())


TIPO_AUTOMOTOR
MOTOCICLETA    244
PEATÓN         222
BICICLETA       86
AUTO            15
CAMIÓN           9
BUS              7
OTRO             6
Name: count, dtype: int64


In [253]:
df[df['TIPO_AUTOMOTOR'].str.contains('VOLQUETA', na=False)]

,MES FALLECIMIENTO,MES ACCIDENTE,BARRIO,COM,PROFESION,SEXO,EDAD,EDAD AGRUPADA,FECHA ACCIDENTE,FECHA FALLECIMIENTO,HORA FALLECIMIENTO,HORA ACCIDENTE,DIA SEMANA FALLECIMIENTO,DIA SEMANA ACCIDENTE,LUGAR INSP.,CONDICCION,VEHICULOS,Ciudad,coordenada X (metros),coordenada Y (metros),coordenada X (km),coordenada Y (km),AÑO_DATA,TIPO_AUTOMOTOR


In [254]:
df[df['TIPO_AUTOMOTOR'].str.contains('CAMIÓN', na=False)]

,MES FALLECIMIENTO,MES ACCIDENTE,BARRIO,COM,PROFESION,SEXO,EDAD,EDAD AGRUPADA,FECHA ACCIDENTE,FECHA FALLECIMIENTO,HORA FALLECIMIENTO,HORA ACCIDENTE,DIA SEMANA FALLECIMIENTO,DIA SEMANA ACCIDENTE,LUGAR INSP.,CONDICCION,VEHICULOS,Ciudad,coordenada X (metros),coordenada Y (metros),coordenada X (km),coordenada Y (km),AÑO_DATA,TIPO_AUTOMOTOR
43,FEBRERO,FEBRERO,COMUNEROS,15,AGRICULTOR,M,15.0,15-19,2009-02-26,2009-02-27,07:00:00,13:00:00,VIERNES,JUEVES,H.U.V.,PAS VOLQUETA,VOLQUETA - CAIDA OCUPANTE,cali,1.065071e+06,869227.197201,1065.071189,869.227197,2009,CAMIÓN
94,ABRIL,ABRIL,ARBOLEDAS,2,CONSTRUCTOR,M,66.0,65-69,2009-04-25,2009-04-27,17:00:00,22:00:00,LUNES,SABADO,H.U.V.,PAS CAMPERO,CAMPERO - CAIDA,cali,1.058948e+06,872917.687266,1058.947793,872.917687,2009,CAMIÓN
137,JUNIO,JUNIO,ALFONSO LOPEZ,7,CONDUCTOR,M,37.0,35-39,2009-06-11,2009-06-11,16:00:00,16:00:00,JUEVES,JUEVES,VIA,CONDUCTOR VOLQUETA,VOLQUETA,cali,1.066039e+06,875000.968639,1066.038617,875.000969,2009,CAMIÓN
254,OCTUBRE,OCTUBRE,TERRON COLORADO,1,JARDINERO,M,38.0,35-39,2009-10-28,2009-10-29,18:00:00,06:00:00,JUEVES,MIERCOLES,CL. ROSARIO,PAS CAMPERO,CAMPERO - VOLCAMIENTO,cali,1.058042e+06,873521.742979,1058.041896,873.521743,2009,CAMIÓN
308,DICIEMBRE,DICIEMBRE,LA MARIA,22,AMA DE CASA,F,58.0,55-59,2009-12-17,2009-12-17,14:00:00,13:00:00,JUEVES,JUEVES,VALLE LILI,PAS CAMPERO,CAMPERO - VOLCAMIENTO,cali,1.059837e+06,857871.436063,1059.837258,857.871436,2009,CAMIÓN
29,FEBRERO,FEBRERO,LA FLORA,2,EMPLEADA,F,33.0,30-34,2010-02-13,2010-02-13,16:10:00,14:50:00,SABADO,Sabado,CL. REMEDIOS,PAS CAMPERO,CAMPERO - OBJETO FIJO (ARBOL),cali,1.062905e+06,877752.823344,1062.905400,877.752823,2010,CAMIÓN
77,ABRIL,ABRIL,EL DIAMANTE,13,RECOLECTOR DE BASURA,M,57.0,55-59,2010-04-14,2010-04-14,06:15:00,06:00:00,MIERCOLES,Miercoles,CL. ROSARIO,PAS CAMION,BUS - CAMION RECOLECTOR EMSIRVA,cali,1.063470e+06,870123.070259,1063.469649,870.123070,2010,CAMIÓN
80,ABRIL,ABRIL,SAN ANTONIO,3,EMPLEADO,M,27.0,25-29,2010-04-17,2010-04-17,05:45:00,05:45:00,SABADO,Sabado,VIA,PAS CAMPERO,CAMPERO - OBJETO FIJO (ARBOL),cali,1.059696e+06,873280.132125,1059.695887,873.280132,2010,CAMIÓN
186,AGOSTO,AGOSTO,ALIRIO M. BELTRAN,14,PINTOR,M,40.0,40-44,2010-08-25,2010-08-25,18:30:00,17:40:00,MIERCOLES,Miercoles,H.U.V.,PAS CAMPERO,CAMPERO - CAMION,cali,1.066352e+06,872296.894232,1066.351722,872.296894,2010,CAMIÓN


---

#### **Exportación de Datos**

In [255]:
# Guardado de Datos
df.to_csv('data/output/Accidentes_2009_2010.csv', index=False)